In [1]:
!curl --header "Host: storage.googleapis.com" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.76" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-US,en;q=0.9" "https://storage.googleapis.com/wandb_datasets/nature_12K.zip" -L -o "nature_12K.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3639M  100 3639M    0     0  56.2M      0  0:01:04  0:01:04 --:--:-- 39.5M


In [ ]:
!unzip "/content/nature_12K.zip"

In [3]:
import shutil
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPooling2D,Activation,Dropout,Flatten,BatchNormalization
from tensorflow.keras.models import Model,Sequential
import random 
from keras.preprocessing.image import ImageDataGenerator
from keras import applications

In [4]:
class CNN:
  def __init__(self,no_of_filters=16,size_of_filters=(3,3),activation_func="relu",neurons_in_dense_layer=128
               ,dropout=0.3,batch_norm=False,doubling=False,augmentation=False):
    self.augmentation = augmentation
    self.dropout = dropout
    self.doubling = doubling
    self.batch_norm = batch_norm
    self.no_of_filters = no_of_filters
    self.size_of_filters = size_of_filters
    self.activation_func = activation_func
    self.neurons_in_dense_layer = neurons_in_dense_layer
    self.train_path = "/content/inaturalist_12K/train/"
    self.cv_path = "/content/inaturalist_12K/cv/"
    self.test_path = "/content/inaturalist_12K/val/"
    
  def generate_data(self):
    if self.augmentation:
      train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True)
    else:
      train_datagen = ImageDataGenerator(rescale=1./255)
    cv_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    self.train_generator = train_datagen.flow_from_directory(self.train_path,target_size=(224,224),seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")
    self.cv_generator = cv_datagen.flow_from_directory(self.cv_path,target_size=(224,224),seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")
    self.test_generator = test_datagen.flow_from_directory(self.test_path,target_size=(224,224),seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")

  def create_cv_data(self):
    classes = sorted(os.listdir(self.train_path))[1:]
    os.mkdir(self.cv_path)
    for cls in classes:
      os.mkdir(self.cv_path+cls)

    for cls in classes:
      no_of_files = len(os.listdir(self.train_path+cls))
      files = os.listdir(self.train_path+cls)
      files = np.asarray(files)

      x = [i for i in range(no_of_files)]
      random.shuffle(x)
      index = x[:int(0.1*no_of_files)]
      file_names = files[index]

      for z in file_names:
        shutil.move(self.train_path+cls+"/"+z , self.cv_path+cls+"/"+z)

  def create_model(self):
    self.generate_data()
    model = Sequential()
    random.seed(10)
    for i in range(5):
      if i==0:
        model.add(Conv2D(self.no_of_filters, self.size_of_filters ,padding="same",activation= self.activation_func, input_shape=(224,224,3)))
      else:
        if self.doubling:
          model.add(Conv2D(self.no_of_filters*(2**i), self.size_of_filters ,padding="same",activation = self.activation_func))
        else:
          model.add(Conv2D(self.no_of_filters, self.size_of_filters ,padding="same",activation = self.activation_func))
      
      if i!=0 and i%2 == 0 and self.batch_norm:
        model.add(BatchNormalization())
      model.add(MaxPooling2D(pool_size=(2, 2))) 

    model.add(Flatten())
    model.add(Dropout(self.dropout))
    model.add(Dense(self.neurons_in_dense_layer,activation = self.activation_func))
    if self.batch_norm:
      model.add(BatchNormalization())
    model.add(Dropout(self.dropout))
    model.add(Dense(10,activation='softmax'))
    return model


In [ ]:
cnn = CNN(no_of_filters=32,neurons_in_dense_layer=128,dropout =0.3
              ,batch_norm=True,doubling=False,augmentation=False)
cnn.create_cv_data()
model = cnn.create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0004),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(cnn.train_generator, validation_data = cnn.cv_generator,epochs=7)

Epoch 1/7
141/141 [==============================] - 242s 2s/step - loss: 2.9382 - accuracy: 0.1459 - val_loss: 2.4444 - val_accuracy: 0.1130
Epoch 2/7
141/141 [==============================] - 209s 1s/step - loss: 2.3702 - accuracy: 0.2140 - val_loss: 2.6662 - val_accuracy: 0.1210
Epoch 3/7
141/141 [==============================] - 212s 2s/step - loss: 2.1885 - accuracy: 0.2545 - val_loss: 2.3246 - val_accuracy: 0.1430
Epoch 4/7
141/141 [==============================] - 210s 1s/step - loss: 2.0989 - accuracy: 0.2754 - val_loss: 2.0707 - val_accuracy: 0.2740
Epoch 5/7
141/141 [==============================] - 212s 2s/step - loss: 2.0587 - accuracy: 0.2891 - val_loss: 2.1270 - val_accuracy: 0.2950
Epoch 6/7
141/141 [==============================] - 209s 1s/step - loss: 2.0219 - accuracy: 0.3017 - val_loss: 2.0180 - val_accuracy: 0.3100
Epoch 7/7
141/141 [==============================] - 208s 1s/step - loss: 1.9575 - accuracy: 0.3251 - val_loss: 2.0081 - val_accuracy: 0.3120


In [ ]:
model.save('best_model.h5')

In [7]:
model = tf.keras.models.load_model('best_model.h5')

In [8]:
model.evaluate(cnn.test_generator)

32/32 [==============================] - 25s 775ms/step - loss: 1.8993 - accuracy: 0.3535


[1.899297833442688, 0.35350000858306885]

In [ ]:
!pip install --upgrade wandb
import wandb
!wandb login 3c967c63b099a3b2acd600aa30008e7de1ea6498
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 11.4MB/s 
     |████████████████████████████████| 133kB 40.1MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 163kB 37.1MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=b097edbb5fcc0ea78ae265f51957134f579028451d55dfb5a3c33b5511e6b664
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=7d528904a2a2a02fd5d77975df0b77e9a5bf1e9e7153f35227815e434a28d833
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# **Question 4(b)**

In [ ]:
wandb.init(project='Assignment 2',entity='adi00510')

wandb: Currently logged in as: adi00510 (use `wandb login --relogin` to force relogin)


In [ ]:
classes=["Amphibia","Animalia","Arachnida","Aves","Fungi","Insecta","Mammalia","Mollusca","Plantae","Reptilia"]
images=[]
labels=[]
x,y = cnn.test_generator.next()
for i in range(0,30):
    temp = x[i]
    temp= np.expand_dims(temp, axis=0)
    images.append(temp)
    labels.append(classes[np.argmax(model.predict(temp))])

In [ ]:
wandb.log({"Question 4": [wandb.Image(img, caption=lbl) for img,lbl in zip(images,labels)]})

# **Question 4(c)**

In [ ]:
wandb.init(project='Assignment 2',entity='adi00510')

_runtime,46
_timestamp,1618416400
_step,0


_runtime,▁
_timestamp,▁
_step,▁


In [9]:
images = []
labels=[]

for i in range(model.weights[0].shape[3]):
    w = model.weights[0][:,:,:,i].numpy()
    for i in range(3):
        w[i] = (w[i] - w[i].min())/(w[i].max() - w[i].min())
    w = w/np.linalg.norm(w)
    images.append(w)
    temp="kernel-"+str(i)
    labels.append(temp)

wandb.log({"Question 4C": [wandb.Image(img, caption=lbl) for img,lbl in zip(images,labels)]})   